# Import Packages

In [2]:
import time

import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import nengo
import numpy as np
import PIL.Image

from collections import deque
from xml.etree import ElementTree as et

from alive_progress import alive_bar
# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib
from dm_control import composer, mjcf, mujoco
from dm_control.utils import inverse_kinematics as ik
from IPython.display import HTML

%env MUJOCO_GL=egl
%matplotlib notebook

env: MUJOCO_GL=egl


# Define Utilities and Parameters

In [3]:
# Rendering parameters
dpi = 100
framerate = 60 # (Hz)
width, height = 720, 480
sensor_shape = (10, 10)

# IK solver parameters
_MAX_STEPS = 50
_TOL = 1e-12
_TIME_STEP = 2e-3


def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())


def get_jacobian(physics, site_name):
    phys = mujoco.Physics.from_model(physics.model)
    jac_pos = np.zeros((3, phys.model.nv))
    jac_rot = np.zeros((3, phys.model.nv))
    mjlib.mj_jacSite(
        phys.model.ptr,
        phys.data.ptr,
        jac_pos,
        jac_rot,
        phys.model.name2id(site_name, 'site'))
    
    return jac_pos, jac_rot

# Modify Taxel Sites in The Scene XML File (if necessary)
* [Mujoco XML Reference](https://mujoco.readthedocs.io/en/latest/XMLreference.html?highlight=site#body-site)
* [The ElementTree XML API](https://docs.python.org/3/library/xml.etree.elementtree.html)

In [76]:
# Define taxel sites to place touch sensors
_N = 10
site_type = 'capsule'
dx = 3e-3
offset = dx*(_N - 1)/2.0

robot_xml = 'models/panda_nohand.xml'
tree = et.parse(robot_xml)
root = tree.getroot()
for body in root.findall('.//body'):
    if body.get('name') == 'taxel_sites':
        sites = list(body.iter('site'))
        for i, site in enumerate(sites):
            site.attrib['type'] = site_type
            site.attrib['size'] = '1e-3 2e-3'
            site.attrib['pos'] = '{:.5f} -.003 {:.5f}'.format(dx*(i//_N)-offset, dx*(i%_N)-offset) # x-z-y axis
            site.attrib['euler'] = '1.570796 0 0'
tree.write(robot_xml) # Save robot XML

## Tactile Sensor
### Soft Sensor Pad (array of balls)
* _M: (x/y-count, z-count)
### Taxel Sites
* site_type: [sphere, capsule, ellipsoid, cylinder, box], “sphere”
* dx: the distance between each site
* offset: uniform distance to shift all sites

In [36]:
# Define 3D arrays of balls connected with sliders
_M = (10, 2)
radius = 5e-4
mass = 1e-6/np.prod(_M)
dx = (3e-2-2*radius)/(_M[0] - 1)
offset = dx*(_M[0] - 1)/2.0
geom_type = 'sphere'
site_type = 'capsule'

robot_xml = 'models/panda_nohand.xml'
tree = et.parse(robot_xml)
root = tree.getroot()
for body in root.findall('.//body'):
    if body.get('name') == 'sensor_pad':
        sensor_pad = body

# If balls are already defined, remove them first
if sensor_pad.find('body') is not None:
    for body in sensor_pad.findall('body'):
        sensor_pad.remove(body)

# Define balls with sliders and touch sensors
sid = 0
for i in range(_M[0]):
    for j in range(_M[0]):
        sid += 1
        # Create base ball
        elem = et.Element('body')
        elem.set('name', 'taxel' + str(sid))
        elem.set('pos', '{:.6f} {:.6f} {:.6f}'.format(dx*i-offset, dx*j-offset, 0))
        inertia = et.Element('inertial')
        inertia.set('mass', str(mass))
        inertia.set('pos', '0 0 0')
        inertia.set('fullinertia', '1e-6 1e-6 1e-6 0 0 0')
        joint = et.Element('joint')
        joint.set('type', 'slide')
        joint.set('axis', '0 0 1')
        joint.set('springdamper', '1e-2 0.5')
        geom = et.Element('geom')
        geom.set('name', 'ball' + str(sid))
        geom.set('type', geom_type)
        geom.set('size', str(radius))
        geom.set('class', 'collision')
        site = et.Element('site')
        site.set('name', 'tendon_site' + str(sid))
        site.set('type', 'sphere')
        site.set('size', '1e-4')
        elem.append(inertia)
        elem.append(joint)
        elem.append(geom)
        elem.append(site)
        sensor_pad.append(elem)

tendon = root.find('tendon')
for spatial in tendon.findall('spatial'):
        tendon.remove(spatial)
for m in range(_M[0]):
    spatial = et.Element('spatial')
    spatial.set('name', 'tendom'+str(m+1))
    spatial.set('width', '2e-4')
    spatial.set('limited', 'true')
    spatial.set('range', '0 {:.6f}'.format(2*dx))
    spatial.set('stiffness', '1.0')
    for n in range(_M[0]):
        site = et.Element('site')
        site.set('site', 'tendon_site' + str(n + 1 + m*_M[0]))
        spatial.append(site)
    tendon.append(spatial)

tree.write(robot_xml) # Save robot

## Key Frames
* Initial poses

In [195]:
# Modify the keyframe home key qpos
njoints = 110
scene_xml = 'models/scene.xml'
tree = et.parse(scene_xml)
root = tree.getroot()
key = root.find('.//keyframe/key')
qpos_value = '0 0 0 -1.570796 0 1.570796 0 0 0 0'
if njoints > 10:
    qpos_value += ' '  + ' '.join(['0' for i in range(njoints - 10)])
key.attrib['qpos'] = qpos_value
tree.write(scene_xml)

# Run Simulations

In [4]:
# Load scene
scene_xml = 'models/scene.xml'
physics = mujoco.Physics.from_xml_path(scene_xml)

# Define simulation variables
site_name = 'attachment_site'
control_site = physics.data.site(name=site_name)
target_site = physics.data.site(name='reach_site3')
joint_names = ['joint{}'.format(i+1) for i in range(7)]
ctrl = np.zeros(10)
duration = 3.0 # (seconds)
omega = np.pi/2 # Rotator angular velocity
video = []
stream = []

# Simulate, saving video frames
physics.reset(0)
ctrl[:7] = physics.data.qpos[:7]
xpos = control_site.xpos
with alive_bar(int(duration/_TIME_STEP) + 1, force_tty=True, title='Rendering') as bar:
    while physics.data.time < duration:
        if np.linalg.norm(control_site.xpos - target_site.xpos) > 0.2:
            result = ik.qpos_from_site_pose(
              physics=physics,
              site_name=site_name,
              target_pos=target_site.xpos,
              target_quat=[0, 0, 0, 1],
              joint_names=joint_names,
              tol=_TOL,
              max_steps=_MAX_STEPS,
              inplace=False,
            )
            if result.success:
                ctrl[:7] = result.qpos[:7]
        ctrl[-3:] = np.array([1, 1, 1])*omega*physics.data.time
        physics.set_control(ctrl)
        physics.step()
    
        data = physics.data.sensordata
        stream.append(data)
        # Save video frames and sensor data
        if len(video) < physics.data.time * framerate:
            pixels = physics.render(camera_id='prospective', width=width, height=height)
            video.append(pixels.copy())
        
        # Update progress 
        bar()

# PIL.Image.fromarray(image)
display_video(video, framerate)

Rendering |████████████████████████████████████████| 1501/1501 [100%] in 17.6s (85.18/s)                                |█▏                                      | ▂▄▆ 42/1501 [3%] in 10s (4.3/s, eta: 5:37) |█▏                                      | ▃▅▇ 43/1501 [3%] in 10s (4.3/s, eta: 5:37) 


In [211]:
forces = []
for cid, contact in enumerate(physics.data.contact):
    force = physics.data.contact_force(cid)
    forces.append(force)
    
len(forces)

39